In [42]:
import nltk
import string
from nltk.corpus import stopwords
from nltk.tag import CRFTagger
import pycrfsuite
from nltk import word_tokenize,sent_tokenize
import pandas as pd
import numpy as np
import spacy

#Import PyTorch Framework
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [43]:
ct = CRFTagger()
nlp = spacy.load('en')

In [44]:
# Read in the train set data
#train_set  = pd.read_csv('Train_and_test_data/train.txt',sep=' ',names=['word','Brill','tag']).drop('Brill',1)
# Format train set data to tuples of a list of lists
#train_set = [[tuple(x) for x in train_set.values]]

In [45]:
#Training Code, Keep it commented unless retraining
#ct.train(train_set,'model.crf.tagger') 

#sentence = "Once upon a time there was a little girl called Cinderella. Cinderella is met a fairy who grants her a wish to be a Princess. Fairy said to Cinderella that she will become a Princess for one night"
sentence = "Once upon a time there was a little girl called Alice . One day Alice met a Rabbit . The Rabbit said to Alice , follow me Alice I will carry you to my Den in SriLanka"


In [46]:
def process_text(passage):
    """
    1.Remove Punctuations and Special characters appear in book chapters
    2.Remove Stopwords
    3.Return Clean list of words
    """
    exclude_set = set(['“','”',':'])

    no_punctuation = [char for char in passage if char not in string.punctuation + "“”."]
    no_punctuation = ''.join(no_punctuation)
    no_punctuation = [word for word in no_punctuation.split() if word.lower() not in stopwords.words('english')]
    
    return no_punctuation

In [47]:
#remove stop words and punctuation
sentence = process_text(sentence)
sentence = ' '.join(sentence)
print(sentence)

upon time little girl called Alice One day Alice met Rabbit Rabbit said Alice follow Alice carry Den SriLanka


In [48]:
sentences = sent_tokenize(sentence)
word_list = [[]]
word_list.clear()
wlist = []
i = 0
j = 0

for sent in sentences:
    for word in sent.split():
        wlist.append(word)
        j += 1
        
    wlist.copy
    word_list.append(wlist.copy())
    wlist.clear()
    i += 1
    
print(word_list)

[['upon', 'time', 'little', 'girl', 'called', 'Alice', 'One', 'day', 'Alice', 'met', 'Rabbit', 'Rabbit', 'said', 'Alice', 'follow', 'Alice', 'carry', 'Den', 'SriLanka']]


In [49]:
#Set the model of previously trained data set
ct.set_model_file(model_file='model.crf.tagger')
word_list = ct.tag_sents(word_list)

#test_set = pd.read_csv('Train_and_test_data/test.txt', sep=' ', names=['words','Brill','tag'], index_col=False).drop(['Brill','tag'],1)
#test_set = test_set.values.tolist()
#ct.tag_sents(test_set)


In [50]:
#test_setEval = pd.read_csv('Train_and_test_data/test.txt', sep=' ', names=['words','Brill','tag']).drop('Brill',1)
#test_setEval = [[tuple(x) for x in test_setEval.values]]
#test the accuracy of the POS
#ct.evaluate(test_setEval)

In [51]:
#Convert list of tuples to a numpy array
word_list = np.array(word_list)

word_list = np.reshape(word_list, (-1,2))

word_list

array([['upon', 'B-PP'],
       ['time', 'B-NP'],
       ['little', 'I-NP'],
       ['girl', 'I-NP'],
       ['called', 'B-VP'],
       ['Alice', 'B-NP'],
       ['One', 'I-NP'],
       ['day', 'I-NP'],
       ['Alice', 'I-NP'],
       ['met', 'I-NP'],
       ['Rabbit', 'I-NP'],
       ['Rabbit', 'I-NP'],
       ['said', 'B-VP'],
       ['Alice', 'B-NP'],
       ['follow', 'I-NP'],
       ['Alice', 'I-NP'],
       ['carry', 'I-NP'],
       ['Den', 'I-NP'],
       ['SriLanka', 'I-NP']], 
      dtype='<U8')

In [52]:
# Find S -> NP VP 
# Returns if a subject is found
# pattern variable = [pat0, pat1, pat2]
def findSubject(pattern):
    #print('{} -> {} {}'.format(pattern[0], pattern[1], pattern[2]))
    if pattern[0] == 'NP' and pattern[1] == 'VP' and pattern[2] == 'NP':
        #if true pattern[0] is the subject of a sentence
        return True
    else:
        return False

In [53]:
#Extract the Subject -> NP VP NP to find the subjects in a sentence
#using Markov Chain Model

pattern = []
subjects = []
word_list_length = len(word_list)

for i in range(0, len(word_list)):
    #print(word_list[i][1].split('-')[1])
    try:
        if((word_list_length - i == 2)):
            break
        else:
            pattern = [word_list[i][1].split('-')[1], word_list[i+1][1].split('-')[1], word_list[i+2][1].split('-')[1]]
            # Call the function findSubject to identify potential subject elements and save them in an array
            if findSubject(pattern):
                # If returns true consider 1st element as a potential Subject
                print('Potential Subject Found at index {}'.format(i))
                print('Subject -> {} => {} {}'.format(word_list[i][0], word_list[i+1][0], word_list[i+2][0]))
                # Put the phrases in to a sentece and append it to an array.
                sub = [word_list[i][0],word_list[i+1][0],word_list[i+2][0]]
                subjects.append(' '.join(sub))
                print('--------------------')
            
    except IndexError:
        # Break the loop if IndexError occurs
        # Fail safe
        print("Out of Index")
        break;

Potential Subject Found at index 3
Subject -> girl => called Alice
--------------------
Potential Subject Found at index 11
Subject -> Rabbit => said Alice
--------------------


In [54]:
# Find PERSONs in the filtered sentence using spaCy NER and build a DataFrame Object based on the data
# DataFrame structure -> Word | POS_TAG | Person

#Convert the subjects array to a 2D numpy array
subjects = np.array(subjects)
subjects = np.reshape(subjects, (-1,1))

refferingDataFrame = pd.DataFrame(columns=('Word','POS_TAG','Person'))

for i in range(0,len(subjects)):
    for j in range(0,len(subjects[i][0].split(" "))):
        wordBag = subjects[i][0].split(" ")
        # Find POS_TAG 
        pos = ct.tag([wordBag[j]])
        
        # Find IF word is Person
        person = [ent.label_ for ent in nlp(pos[0][0]).ents]
        person = ' '.join(person)
        if not person:
            person = ""
            
        #Add a row to the DataFrame with the retrived data
        refferingDataFrame.loc[len(refferingDataFrame)] = [pos[0][0], pos[0][1].split('-')[1], person]
        
refferingDataFrame

,Word,POS_TAG,Person
0,girl,ADJP,
1,called,VP,
2,Alice,NP,PERSON
3,Rabbit,NP,PERSON
4,said,VP,
5,Alice,NP,PERSON


In [55]:
# Create a LUT based on the verb phrase and the Subject phrase of the sentence to make the question
# Create a Template to the question to be generated

In [56]:
LUTdataFrame = pd.read_csv('LUT.csv')
LUTdataFrame

,Word_Phrase,Question_Phrase
0,NP PERSON,Who is $
1,VP,What did $ $ $


In [57]:
# Based on the data in refferingDataFrame and LUTdataFrame create appropriate qusetions
# eg:- if 'Alice' = NP & Person then, "Who is "+ Alice(NP)
# eg:- if 'called' '= VP then, "What did " + Rabbit(NP) said(VP) Alice(NP)
 
for index,row in refferingDataFrame.iterrows():
    testCase = (row['POS_TAG'] + " " + row['Person']).rstrip()
    for lineNo,comparingRow in LUTdataFrame.iterrows():
        # Check how many reference points in the LUT table senetence
        if(testCase == comparingRow['Word_Phrase']):
            #Generate Sentence according the Word_Phrase
            Question = comparingRow['Question_Phrase'] + " " + row['Word'] + " ?"
            print(Question)

What did $ $ $ called ?
Who is $ Alice ?
Who is $ Rabbit ?
What did $ $ $ said ?
Who is $ Alice ?
